In [3]:
with open('harrypotter.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  25899


In [5]:
chars = sorted(list(set(text)))
vocab_size =len(chars)
print(''.join(chars))
print(vocab_size)


 !"'(),-.0123456789:;?ABCDEFGHIJKLMNOPRSTUVWYabcdefghijklmnopqrstuvwxyz
72


In [6]:
#converting characters into tokens by using character level tokenizer this means we use small codebooks and simple encode decode function also small vocabulary and long sequences
#so tokenizers convert character into integers and then again into character and they have a limited length of integers. we can have small vocablury size and long sequence of integers and large vocabulary and small sequence of integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s:[stoi[c] for c in s]#encoder converts text to integer
decode = lambda l: ''.join([itos[i] for i in l])#decoder converts integer to text

print(encode("hello how are you"))
print(decode(encode("hello how are you")))

[53, 50, 57, 57, 60, 1, 53, 60, 68, 1, 46, 63, 50, 1, 70, 60, 66]
hello how are you


In [7]:
# here we are tokenizing the whole dataset using pytorch library
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.type)
print(data[:1000])# 1000 indicates first 1000 characters from dataset 

torch.Size([25899]) <built-in method type of Tensor object at 0x000002349B905850>
tensor([30, 46, 63, 63, 70,  1, 38, 60, 65, 65, 50, 63,  1, 46, 59, 49,  1, 65,
        53, 50,  1, 40, 60, 63, 48, 50, 63, 50, 63,  4, 64,  1, 40, 65, 60, 59,
        50,  0, 25, 30, 23, 38, 41, 27, 39,  1, 37, 36, 27,  0, 41, 30, 27,  1,
        24, 37, 45,  1, 44, 30, 37,  1, 34, 31, 43, 27, 26,  0, 35, 63,  9,  1,
        46, 59, 49,  1, 35, 63, 64,  9,  1, 26, 66, 63, 64, 57, 50, 70,  7,  1,
        60, 51,  1, 59, 66, 58, 47, 50, 63,  1, 51, 60, 66, 63,  7,  1, 38, 63,
        54, 67, 50, 65,  1, 26, 63, 54, 67, 50,  7,  1, 68, 50, 63, 50,  1, 61,
        63, 60, 66, 49,  1, 65, 60,  1, 64, 46, 70,  0, 65, 53, 46, 65,  1, 65,
        53, 50, 70,  1, 68, 50, 63, 50,  1, 61, 50, 63, 51, 50, 48, 65, 57, 70,
         1, 59, 60, 63, 58, 46, 57,  7,  1, 65, 53, 46, 59, 56,  1, 70, 60, 66,
         1, 67, 50, 63, 70,  1, 58, 66, 48, 53,  9,  1, 41, 53, 50, 70,  1, 68,
        50, 63, 50,  1, 65, 53, 50,  1

In [8]:
#we are splitting our data into training(90%) and testing(10%) sets. This will help us to what extend our model is overfitting
#so we will hide and keep the validation data on the side
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
 # now we will plug the data into transformer and we are not going to feed the entire data all at once into transformer because it will be computationally heavy so we will use chunks of datasets
# we will define the maximum length of these chunks called block size
block_size = 10
train_data[:block_size+1]

tensor([30, 46, 63, 63, 70,  1, 38, 60, 65, 65, 50])

In [10]:
# this block size has 11 individual examples packed in there which is learning what comes next with every next exmaple
# x is the input to the transformer, first block size character 
x= train_data[:block_size]
y = train_data[1:block_size+1]# y will be the next block size character so its offset by 1. y is the target for each position in inputs
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([30]) the target: 46
when input is tensor([30, 46]) the target: 63
when input is tensor([30, 46, 63]) the target: 63
when input is tensor([30, 46, 63, 63]) the target: 70
when input is tensor([30, 46, 63, 63, 70]) the target: 1
when input is tensor([30, 46, 63, 63, 70,  1]) the target: 38
when input is tensor([30, 46, 63, 63, 70,  1, 38]) the target: 60
when input is tensor([30, 46, 63, 63, 70,  1, 38, 60]) the target: 65
when input is tensor([30, 46, 63, 63, 70,  1, 38, 60, 65]) the target: 65
when input is tensor([30, 46, 63, 63, 70,  1, 38, 60, 65, 65]) the target: 50


In [11]:
#now we will make batch of chunks for improving efficiency since gpu are good at parallel processing of data
# chunks will be processed simuntaneously but completely independent
torch.manual_seed(1337)
batch_size = 4 #how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):#generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y 

xb, yb = get_batch('train')
print('inputs :')
print(xb.shape)
print(xb)
print('targets: ')
print(yb.shape)
print(yb)

print('-----')

for b in range(batch_size):#batch dimension
    for t in range(block_size):# time dimensions
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")


inputs :
torch.Size([4, 8])
tensor([[50,  1, 68, 53, 60,  1, 65, 60],
        [ 0,  3, 31,  4, 58,  1, 59, 60],
        [64, 65, 46, 65, 66, 50,  7,  1],
        [ 1, 51, 60, 66, 63,  7,  1, 65]])
targets: 
torch.Size([4, 8])
tensor([[ 1, 68, 53, 60,  1, 65, 60, 57],
        [ 3, 31,  4, 58,  1, 59, 60, 65],
        [65, 46, 65, 66, 50,  7,  1, 54],
        [51, 60, 66, 63,  7,  1, 65, 53]])
-----
when input is [50] the target: 1
when input is [50, 1] the target: 68
when input is [50, 1, 68] the target: 53
when input is [50, 1, 68, 53] the target: 60
when input is [50, 1, 68, 53, 60] the target: 1
when input is [50, 1, 68, 53, 60, 1] the target: 65
when input is [50, 1, 68, 53, 60, 1, 65] the target: 60
when input is [50, 1, 68, 53, 60, 1, 65, 60] the target: 57
when input is [0] the target: 3
when input is [0, 3] the target: 31
when input is [0, 3, 31] the target: 4
when input is [0, 3, 31, 4] the target: 58
when input is [0, 3, 31, 4, 58] the target: 1
when input is [0, 3, 31, 4, 58,

In [12]:
print(xb)

tensor([[50,  1, 68, 53, 60,  1, 65, 60],
        [ 0,  3, 31,  4, 58,  1, 59, 60],
        [64, 65, 46, 65, 66, 50,  7,  1],
        [ 1, 51, 60, 66, 63,  7,  1, 65]])


In [13]:
import torch 
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) 

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)


        return logits , loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
        
   
   
m = BigramLanguageModel (vocab_size)
logits, loss = m(xb, yb)
out = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

     

torch.Size([32, 72])
tensor(5.0302, grad_fn=<NllLossBackward0>)

5ydvWCNg8JJSEx0fbonxMfN7NSoTyUznc'5-,bKi6mWzJ:kdWbbu:m-Jhcluex;T0C5J2sCx?g!N-E609ac;N-q;UTtMVu.7o:V5


In [14]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [15]:
batch_size = 32
for steps in range(1000):

    xb, yb = get_batch('train')

    logits, loss =m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())    

3.7530899047851562


In [16]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))



hveTAyIhekOC;UIj0?(
v?Lrj3?i.
 h6M7stJPNoocYpM3(h6SP
kWchs-hFFKs,P25LIdS(rM8)(cx0ueed -nTRz2uOys4crm


In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('harrypotter.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.210632 M parameters
step 0: train loss 4.4233, val loss 4.4327
step 100: train loss 2.6112, val loss 2.6275
step 200: train loss 2.4513, val loss 2.4595
step 300: train loss 2.3434, val loss 2.3721
step 400: train loss 2.2323, val loss 2.2541
step 500: train loss 2.1122, val loss 2.1813
step 600: train loss 2.0245, val loss 2.0859
step 700: train loss 1.9131, val loss 2.0117
step 800: train loss 1.8476, val loss 1.9544
step 900: train loss 1.7742, val loss 1.9091
step 1000: train loss 1.7104, val loss 1.8683
step 1100: train loss 1.6621, val loss 1.8369
step 1200: train loss 1.6198, val loss 1.8345
step 1300: train loss 1.5683, val loss 1.8294
step 1400: train loss 1.5159, val loss 1.7982
step 1500: train loss 1.4702, val loss 1.7815
step 1600: train loss 1.4259, val loss 1.8066
step 1700: train loss 1.3873, val loss 1.8023
step 1800: train loss 1.3488, val loss 1.8030
step 1900: train loss 1.3161, val loss 1.7932
step 2000: train loss 1.2847, val loss 1.7942
step 2100: train loss 1.